# Simple pipeline using a shared queue (SPSC)
First introduce the code for the single producer single consumer queue using condition variables to synchronize accesses

Some includes to get necessary functionalities

In [1]:
#include <iostream>
#include <mutex>
#include <condition_variable>
#include <deque>
#include <vector>
#include <chrono>
#include <cstddef>
#include <math.h>
#include <string>

The queue code itself. It is defined as a template such that we can defined queues with different item types:

In [2]:
template <typename T>
class myqueue
{
private:
  std::mutex              d_mutex;
  std::condition_variable d_condition;
  std::deque<T>           d_queue;
public:

  myqueue(std::string s) { std::cout << "Created " << s << " queue " << std::endl;  }
  myqueue() {}
  
  void push(T const& value) {
    {
      std::unique_lock<std::mutex> lock(d_mutex);
      d_queue.push_front(value);
    }
    this->d_condition.notify_one();
  }
  
  T pop() {
    std::unique_lock<std::mutex> lock(d_mutex);
    d_condition.wait(lock, [=]{ return !d_queue.empty(); });  // wait if the queue is currently empty
    T rc(std::move(this->d_queue.back()));       // used to get the actual message rather than a copy
    d_queue.pop_back();                   // remove the item just read from the back end of the queue
    return rc;
  }
};


Finally we need something to denote the end of a stream, modelled with the queue. We will use queues of positive integeres, therefore let's go for a negative number

In [3]:
#define EOS -1

## Thread to read from the queue

This is the function to read and print something from the queue hosting pointers to integers. It will be used as thread body of the last pipeline stage.
We will use a sleep to slow down queue pushes.

In [4]:
#include <chrono>
#include <thread>
using namespace std::chrono_literals;

In [5]:
void drain(myqueue<int> &q) {
  std::cout << "Drain started" << std::endl;
  auto e = q.pop();
  
  while(e != EOS) {
    std::cout << "received " << e << std::endl;
    e = q.pop();
  }
  return;
}

This is the function to send something to an integer pointer queue (stream creation). It will be used as the thread body for the first pipeline stage

In [6]:
void source(myqueue<int> &q) {
    for(int i=0; i<16; i++){
        q.push(i);
        std::this_thread::sleep_for(1000ms);
    }
    q.push(EOS);
    std::cout << "sent EOS" << std::endl;
    return;
}

Now we create an integer queue:

In [7]:
myqueue<int> myq;

We can start the drain. It will stop immediately blocked on the condition variable, as the queue is still empty:

In [8]:
std::thread tdrain(drain,std::ref(myq));

Drain started


Now we can start the thread filling the queue:

In [9]:
std::thread tsource(source,std::ref(myq));

received 0


Items are received and printed from the drain with the delay of dictated by the source. Let's use a fresh queue:

In [10]:
tsource.join();

received 1
received 2
received 3
received 4
received 5
received 6
received 7
received 8
received 9
received 10
received 11
received 12
received 13
received 14
received 15
sent EOS


In [11]:
tdrain.join();

In [12]:
myqueue<int> myq1;

Now start a new source thread:

In [13]:
std::thread tsource1(source,std::ref(myq1));

Wait a few seconds and start the drain thread:

In [14]:
std::thread tdrain1(drain,std::ref(myq1));

Drain started
received 0
received 1
received 2
received 3
received 4
received 5
received 6
received 7
received 8
received 9
received 10
received 11
received 12
received 13
received 14
received 15
sent EOS


You'll see the items accumulated while waiting to start the second thread are immediately received from drain thread. 

In [15]:
tsource1.join();

In [16]:
tdrain1.join();